# Training a Convolutional Neural Network 

## Load and normalize the CIFAR10 training and test datasets using torchvision



## Define a Convolutional Neural Network


## Define a loss function

## Train the network on the training data

## Test the network on the test data